In [3]:
import pandas as pd
import os
import pymongo
from pymongo import MongoClient
from pymongo import TEXT


XLS_FILE=r'C:\Users\Utilisateur\OneDrive - IDENEA\Agence\Dossiers en cours\ODYCEE - BAYER - PSE - mai 2022\Documents partagés\Echantillonnage liste du personnel pour entretiens V3.xlsx'
DICT_CONNECT1={"database_name" : 'bayer' ,"collection_name" : 'person',"server" : 'localhost',"mongodb_port" : 27018}

class MongoDBConnection:

    def __init__(self,database_name = 'mydatabasename' ,collection_name='mycollectionname',mongodb_port = 27017,server='localhost'):
        self.db=MongoClient(server,int(mongodb_port))[database_name]
        self.collection = self.db[collection_name]


    def filter_mongoDB(self,condition,field):
        result_df=pd.DataFrame(list(self.collection.find(condition,field)))

        return result_df

    def write_df_to_mongoDB(self,df):
        #"""
        #This function take a list and create a collection in MongoDB (you should
        #provide the database name, collection, port to connect to the remoete database,
        #server of the remote database, local port to tunnel to the other machine)
        #
        #---------------------------------------------------------------------------
        #Parameters / Input
        #    my_list: the list to send to MongoDB
        #    database_name:  database name
        #
        #    collection_name: collection name (to create)
        #    server: the server of where the MongoDB database is hosted
        #        Example: server = 'XXX.XXX.XX.XX'
        #    this_machine_port: local machine port.
        #        For example: this_machine_port = '27017'
        #    remote_port: the port where the database is operating
        #        For example: remote_port = '27017'
        #    chunk_size: The number of items of the list that will be send at the
        #        some time to the database. Default is 100.
        #
        #Output
        #    When finished will print "Done"
        #----------------------------------------------------------------------------
        #FUTURE modifications.
        #1. Write to SQL
        #2. Write to csv
        #----------------------------------------------------------------------------
        #30/11/2017: Rafael Valero-Fernandez. Documentation
        #"""

        # To write
        self.collection.delete_many({})  # Destroy the collection
        #aux_df=aux_df.drop_duplicates(subset=None, keep='last') # To avoid repetitions
        my_list = df.to_dict('records')
        # Inser chunks of the dataframe
        list_chunked = [my_list[i:i + chunk_size] for i in range(0, len(my_list), chunk_size)]

        for chunk in list_chunked:
            #print ("chunk :",chunk)
            collection.insert_many(chunk) # fill de collection

        print('Done')
        return


    def text_search(self,text_search):
        # set connection 

        for index in self.collection.list_indexes():
            print(index)
        self.collection.create_index([("$**",pymongo.TEXT)],name='search_index', default_language='french')
        #condition={ $text: { $search: text_search }}

        results=self.collection.find({"$text": { "$search": text_search }})

        result_df=pd.DataFrame(list(results))

        return result_df


    def write_xlsx_to_mongoDB(self,xls_file):
        df=pd.DataFrame()
        df=pd.read_excel(xls_file)
        #display(df)
        self.write_df_to_mongoDB(df)

        return True

    def get_field_list(self):
  
        cursor = self.collection.find({})
        field_list=[]

        first_row = next(cursor, None)
        if first_row:
            field_list=list(first_row.keys())

        for document in cursor: 
            field_list=list(set(field_list)& set(list(document.keys())))
        # print(field_list)
            # print all fields of this document. 
        return field_list




def main():

    bayer=MongoDBConnection(**DICT_CONNECT1)
    print(bayer.text_search("assistante"))

"""  
insert_datas_from_excel(XLS_FILE,**DICT_CONNECT)

    get_field_list(**DICT_CONNECT)

    results= filter_mongoDB({"service":"Ventes Mktg PH"},{},**DICT_CONNECT)

    results=text_search_mongoDB({ "nom": "text" },'',**DICT_CONNECT)
    results.set_index('nom',inplace=True)
    display(results)"""



 
if __name__ == '__main__':
    main()
    
    

['A', 'Poste', 'Age du salarié', 'Select', 'Code PCS : Nomenclature des pr', 'Unnamed: 24', '_id', 'UnitStru', 'Contrat', 'Manager ID', 'Unité structurelle', 'Domaine', 'ManagName', 'VG', 'Sexe', 'prenom', 'Ancienneté en année', 'Désignation entité organisatio', 'Fait/prévu', 'SDENIS', 'Statut', 'Entretien ALB', 'Email', 'Mat.', 'STANNEUX', 'Poste.1', 'service', 'STANNEUX.1', 'R', 'PCS', 'nom']
